Juntar QueroTruck e Grupo Vamos

In [ ]:
import pandas as pd

# Carregar o arquivo Excel
arquivo_excel = 'teste_dados_Truck&Vamos.xlsx'

# Verificar as abas disponíveis no arquivo
excel_file = pd.ExcelFile(arquivo_excel)
print("Abas disponíveis no arquivo:", excel_file.sheet_names)

# Carregar os dados (ajustar com base nas abas disponíveis)
if 'QueroTruck' in excel_file.sheet_names and 'GrupoVamos' in excel_file.sheet_names:
    aba1 = pd.read_excel(arquivo_excel, sheet_name='QueroTruck')
    aba2 = pd.read_excel(arquivo_excel, sheet_name='GrupoVamos')
    dados_combinados = pd.concat([aba1, aba2], ignore_index=True)
else:
    print("Abas 'QueroTruck' e/ou 'GrupoVamos' não encontradas. Carregando a primeira aba disponível.")
    dados_combinados = pd.read_excel(arquivo_excel, sheet_name=excel_file.sheet_names[0])

# Selecionar apenas as colunas esperadas
colunas_esperadas = ['Marca', 'Modelo', 'Preço', 'Quilometragem', 'Ano', 'Localização']
dados_combinados = dados_combinados[colunas_esperadas]

# Função para limpar a coluna 'Localização'
def limpar_localizacao(row):
    # Pegar o valor da coluna 'Localização' e converter para string
    localizacao = str(row['Localização'])
    
    # Se a célula for "nan" ou "Tirar", retornar como está
    if localizacao == 'nan' or localizacao == 'Tirar':
        return localizacao
    
    # Normalizar quebras de linha (substituir \r\n por \n)
    localizacao = localizacao.replace('\r\n', '\n').replace('\r', '\n')
    
    # Dividir o texto em linhas
    linhas = localizacao.split('\n')
    
    # Remover espaços em branco e converter para maiúsculas para verificação
    linhas_upper = [linha.strip().upper() for linha in linhas]
    
    # Depuração: Mostrar o texto original e as linhas processadas
    print(f"Texto original: {localizacao}")
    print(f"Linhas (upper): {linhas_upper}")
    
    # Verificar se "Anunciante" está presente
    if any(linha_upper == 'ANUNCIANTE' for linha_upper in linhas_upper):
        print("Detectado 'Anunciante'")
        novas_linhas = []
        i = 0
        while i < len(linhas):
            if linhas_upper[i] == 'ANUNCIANTE':
                # Pular "Anunciante" e a próxima linha (nome do anunciante), se existir
                print(f"Removendo 'Anunciante' na posição {i}")
                i += 2
            else:
                # Manter a linha se não for "Anunciante"
                if linhas[i].strip():  # Ignorar linhas vazias
                    novas_linhas.append(linhas[i])
                i += 1
        print(f"Após remover 'Anunciante' e nome: {novas_linhas}")
        return '\n'.join(novas_linhas).strip() if novas_linhas else ''
    
    # Verificar se "ABAIXO FIPE" está presente
    if any(linha_upper == 'ABAIXO FIPE' for linha_upper in linhas_upper):
        print("Detectado 'ABAIXO FIPE'")
        novas_linhas = [linha for linha, linha_upper in zip(linhas, linhas_upper) if linha_upper != 'ABAIXO FIPE' and linha.strip() != '']
        print(f"Após remover 'ABAIXO FIPE': {novas_linhas}")
        return '\n'.join(novas_linhas).strip() if novas_linhas else localizacao
    
    # Se não contém "Anunciante" nem "ABAIXO FIPE", retornar o texto original
    print("Nenhum 'Anunciante' ou 'ABAIXO FIPE' detectado")
    return localizacao

# Aplicar a função para limpar a coluna 'Localização'
dados_combinados['Localização'] = dados_combinados.apply(limpar_localizacao, axis=1)

# Salvar o arquivo final
dados_combinados.to_excel('teste_dados_queroTruck_grupoVamos.xlsx', index=False)

Base Camhinhões

In [ ]:
import pandas as pd

truck_vamos = pd.read_excel(r"C:\Users\gabriel.vinicius\Documents\Vscode\Caminhoes\dados_queroTruck_grupoVamos.xlsx")
truncadao = pd.read_excel('dados_trucadao_completos.xlsx')  

preco_sob_consulta = ['PREÇO SOB CONSULTA']
truncadao = truncadao[~truncadao['Preço'].astype(str).str.upper().str.strip().isin(preco_sob_consulta)]

colunas_esperadas = ['Marca', 'Modelo', 'Preço', 'Quilometragem', 'Ano', 'Localização']

truck_vamos = truck_vamos[colunas_esperadas]
truncadao = truncadao[colunas_esperadas]

dados_combinados = pd.concat([truck_vamos, truncadao], ignore_index=True)

dados_combinados = dados_combinados[~dados_combinados.apply(
    lambda row: all(isinstance(x, str) and x.strip().upper() == 'NÃO INFORMADO' for x in row), axis=1)]

dados_combinados.to_excel(r"C:\Users\gabriel.vinicius\Documents\Bases Scraping\Base Caminhão\Base Unificada 26-06-2025.xlsx", index=False)

In [ ]:
import pandas as pd

truncadao

Organizando Truncadão

In [ ]:
import pandas as pd

# Lê o arquivo Excel
arquivo_excel = r"C:\Users\gabriel.vinicius\Documents\Vscode\Caminhoes\dados_trucadao_completos.xlsx"  # Ajuste o caminho se necessário
df = pd.read_excel(arquivo_excel)

def organizar_dados_carros(df):
    # Seleciona apenas as colunas desejadas
    colunas_desejadas = ['Marca', 'Modelo', 'Preço', 'Quilometragem', 'Ano', 'Localização']
    
    # Verifica quais colunas existem no DataFrame e filtra
    colunas_presentes = [col for col in colunas_desejadas if col in df.columns]
    df_organizado = df[colunas_presentes].copy()
    
    # Converte Preço para inteiro, tratando "R$ ", pontos e vírgulas
    if 'Preço' in df_organizado.columns:
        # Substitui "Preço inválido" por NaN e remove "R$ "
        df_organizado['Preço'] = df_organizado['Preço'].replace('Preço inválido', pd.NA)
        # Remove "R$ ", espaços, pontos de milhar e vírgulas
        df_organizado['Preço'] = df_organizado['Preço'].replace(r'[R$\s]', '', regex=True
                                                              ).replace(r'\.', '', regex=True
                                                              ).replace(',', '', regex=True)
        # Converte diretamente para inteiro (Int64 para suportar NaN)
        df_organizado['Preço'] = df_organizado['Preço'].astype('Int64')
    
    # Converte Quilometragem para inteiro, tratando "Não informado"
    if 'Quilometragem' in df_organizado.columns:
        # Substitui "Não informado" por NaN
        df_organizado['Quilometragem'] = df_organizado['Quilometragem'].replace('Não informado', pd.NA)
        # Remove caracteres não numéricos
        df_organizado['Quilometragem'] = df_organizado['Quilometragem'].replace(r'[^\d]', '', regex=True)
        # Converte diretamente para inteiro (Int64 para suportar NaN)
        df_organizado['Quilometragem'] = df_organizado['Quilometragem'].astype('Int64')
    
    # Converte Ano para inteiro (ex: "18/18" -> 2018)
    if 'Ano' in df_organizado.columns:
        def converter_ano(ano):
            if isinstance(ano, str):
                if ano == 'Não informado':
                    return pd.NA
                # Pega o primeiro número do formato "18/18" e converte para 2018
                primeiro_ano = ano.split('/')[0]
                # Se for menor que 100, assume que é 20XX ou 19XX
                try:
                    ano_num = int(primeiro_ano)
                    if ano_num < 100:
                        if ano_num >= 50:
                            return 1900 + ano_num  # Ex: 95/95 -> 1995
                        else:
                            return 2000 + ano_num  # Ex: 18/18 -> 2018
                    return ano_num
                except:
                    return pd.NA
            return ano

        # Aplica a conversão e converte diretamente para inteiro (Int64 para suportar NaN)
        df_organizado['Ano'] = df_organizado['Ano'].apply(converter_ano).astype('Int64')
    
    return df_organizado

# Aplica a função ao DataFrame lido
dados_trucadao_completos = organizar_dados_carros(df)

# Mostra as primeiras linhas do resultado
print(dados_trucadao_completos.head())

# Salva o resultado em um novo arquivo Excel
dados_trucadao_completos.to_excel("dados_organizados.xlsx", index=False)